In [87]:
import pandas as pd
from pandas import json_normalize

In [88]:
# Replace the path below with the actual path to your CSV file if it's not in the same directory as your Jupyter Notebook
file_path = 'commodities_data.csv'

# Read the CSV file
df = pd.read_csv(file_path)

In [89]:
# Convert the datetime column to a datetime object
#df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M')

# Convert the 'datetime' column to datetime objects
df['datetime'] = pd.to_datetime(df['datetime'])

In [90]:
# Display the first few rows of the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46568732 entries, 0 to 46568731
Data columns (total 8 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          int64         
 1   quantity    int64         
 2   unit_price  int64         
 3   time_left   object        
 4   item.id     int64         
 5   item.name   object        
 6   item.class  object        
 7   datetime    datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 2.8+ GB


In [91]:
df.head()

,id,quantity,unit_price,time_left,item.id,item.name,item.class,datetime
0,1899749198,15,109400,SHORT,184090,Potion of the Psychopomp's Speed,Consumables,2023-12-28 15:41:00
1,1899749311,1,2000,SHORT,168650,Cragscale,Trade Goods,2023-12-28 15:41:00
2,1899749353,6,2400,SHORT,152576,Tidespray Linen,Trade Goods,2023-12-28 15:41:00
3,1899749415,2,290200,SHORT,76885,Burning Primal Diamond,Gems,2023-12-28 15:41:00
4,1899749701,1,9707000,SHORT,139362,Glyph of Mana Touched Souls,Glyphs,2023-12-28 15:41:00


In [92]:
# Group by 'item.class' and count unique 'item.id'
class_counts = df.groupby('item.class')['item.id'].nunique()

print("Number of unique items for each item class:")
print(class_counts)

Number of unique items for each item class:
item.class
Armor              11
Consumables      3183
Gems              998
Glyphs            111
Miscellaneous    1912
Recipes             3
Trade Goods      2092
Unknown           550
Name: item.id, dtype: int64


In [93]:
# Count all rows in the DataFrame
total_rows = len(df)
print(f"Total rows in the DataFrame: {total_rows:,}")

Total rows in the DataFrame: 46,568,732


In [94]:
# Replace 'Unknown' with NaN for item.name and item.class
df['item.name'] = df['item.name'].replace('Unknown', pd.NA)
df['item.class'] = df['item.class'].replace('Unknown', pd.NA)

# Count the total number of items
total_items = len(df)

# Count items where 'name' or 'class' is missing or 'Unknown'
missing_name = len(df[df['item.name'].isna()])
missing_class = len(df[df['item.class'].isna()])
missing_both = len(df[df['item.name'].isna() & df['item.class'].isna()])

# Print the results
print(f"Total rows: {total_items:,}")
print(f"Rows missing name: {missing_name:,}")
print(f"Rows missing class: {missing_class:,}")
print(f"Rows missing both name and class: {missing_both:,}")


Total rows: 46,568,732
Rows missing name: 3,238
Rows missing class: 4,434,558
Rows missing both name and class: 3,238


In [69]:
df.head(5)

,id,quantity,unit_price,time_left,item.id,item.name,item.class,datetime
0,1899749198,15,109400,SHORT,184090,Potion of the Psychopomp's Speed,Consumables,2023-12-28 15:41:00
1,1899749311,1,2000,SHORT,168650,Cragscale,Trade Goods,2023-12-28 15:41:00
2,1899749353,6,2400,SHORT,152576,Tidespray Linen,Trade Goods,2023-12-28 15:41:00
3,1899749415,2,290200,SHORT,76885,Burning Primal Diamond,Gems,2023-12-28 15:41:00
4,1899749701,1,9707000,SHORT,139362,Glyph of Mana Touched Souls,Glyphs,2023-12-28 15:41:00


In [86]:
# Filter the DataFrame for item.id 184090
filtered_df = df[df['item.id'] == 184090]

# Group by 'id' and sort each group by 'unit_price' in ascending order
grouped_sorted_df = filtered_df.groupby('id').apply(lambda x: x.sort_values('unit_price', ascending=True)).reset_index(drop=True)
grouped_sorted_df.head(250)

,id,quantity,unit_price,time_left,item.id,item.name,item.class,datetime
0,1877385343,16,85900,MEDIUM,184090,Potion of the Psychopomp's Speed,Consumables,2023-12-22 21:41:00
1,1877385343,16,85900,LONG,184090,Potion of the Psychopomp's Speed,Consumables,2023-12-22 19:41:00
2,1877385343,16,85900,LONG,184090,Potion of the Psychopomp's Speed,Consumables,2023-12-22 20:41:00
3,1877385343,16,85900,LONG,184090,Potion of the Psychopomp's Speed,Consumables,2023-12-22 18:41:00
4,1877385343,16,85900,SHORT,184090,Potion of the Psychopomp's Speed,Consumables,2023-12-22 22:41:00
...,...,...,...,...,...,...,...,...
245,1885816993,3,30000000,VERY_LONG,184090,Potion of the Psychopomp's Speed,Consumables,2023-12-23 19:41:00
246,1885816993,3,30000000,VERY_LONG,184090,Potion of the Psychopomp's Speed,Consumables,2023-12-23 07:41:00
247,1885816993,3,30000000,VERY_LONG,184090,Potion of the Psychopomp's Speed,Consumables,2023-12-23 23:41:00
248,1885816993,3,30000000,LONG,184090,Potion of the Psychopomp's Speed,Consumables,2023-12-24 15:41:00
